<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

<h1 align="center"><font size="5">Classification with Python</font></h1>


In this notebook we try to practice all the classification algorithms that we learned in this course.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Lets first load required libraries:


In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

### About dataset


This dataset is about past loans. The **Loan_train.csv** data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
| -------------- | ------------------------------------------------------------------------------------- |
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |


Lets download the dataset


In [2]:
!wget -O loan_train.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv

--2021-03-02 23:01:39--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/FinalModule_Coursera/data/loan_train.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

loan_train.csv      100%[===================>]  22.56K  --.-KB/s    in 0.001s  

2021-03-02 23:01:40 (16.9 MB/s) - ‘loan_train.csv’ saved [23101/23101]



### Load Data From CSV File


In [3]:
df = pd.read_csv('loan_train.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,9/8/2016,10/7/2016,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,9/8/2016,10/7/2016,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,9/8/2016,9/22/2016,27,college,male
3,4,4,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,college,female
4,6,6,PAIDOFF,1000,30,9/9/2016,10/8/2016,29,college,male


In [4]:
df.shape

(346, 10)

### Convert to date time object


In [5]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,2016-09-08,2016-10-07,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,2016-09-08,2016-10-07,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,2016-09-08,2016-09-22,27,college,male
3,4,4,PAIDOFF,1000,30,2016-09-09,2016-10-08,28,college,female
4,6,6,PAIDOFF,1000,30,2016-09-09,2016-10-08,29,college,male


# Data visualization and pre-processing


Let’s see how many of each class is in our data set 


In [6]:
df['loan_status'].value_counts()

PAIDOFF       260
COLLECTION     86
Name: loan_status, dtype: int64

260 people have paid off the loan on time while 86 have gone into collection 


Lets plot some columns to underestand data better:


In [7]:
# notice: installing seaborn might takes a few minutes
!conda install -c anaconda seaborn -y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ ^C
| 

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction


### Lets look at the day of the week people get the loan


In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()


We see that people who get the loan at the end of the week dont pay it off, so lets use Feature binarization to set a threshold values less then day 4 


In [8]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender,dayofweek,weekend
0,0,0,PAIDOFF,1000,30,2016-09-08,2016-10-07,45,High School or Below,male,3,0
1,2,2,PAIDOFF,1000,30,2016-09-08,2016-10-07,33,Bechalor,female,3,0
2,3,3,PAIDOFF,1000,15,2016-09-08,2016-09-22,27,college,male,3,0
3,4,4,PAIDOFF,1000,30,2016-09-09,2016-10-08,28,college,female,4,1
4,6,6,PAIDOFF,1000,30,2016-09-09,2016-10-08,29,college,male,4,1


## Convert Categorical features to numerical values


Lets look at gender:


In [9]:
df.groupby(['Gender'])['loan_status'].value_counts(normalize=True)

Gender  loan_status
female  PAIDOFF        0.865385
        COLLECTION     0.134615
male    PAIDOFF        0.731293
        COLLECTION     0.268707
Name: loan_status, dtype: float64

86 % of female pay there loans while only 73 % of males pay there loan


Lets convert male to 0 and female to 1:


In [10]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender,dayofweek,weekend
0,0,0,PAIDOFF,1000,30,2016-09-08,2016-10-07,45,High School or Below,0,3,0
1,2,2,PAIDOFF,1000,30,2016-09-08,2016-10-07,33,Bechalor,1,3,0
2,3,3,PAIDOFF,1000,15,2016-09-08,2016-09-22,27,college,0,3,0
3,4,4,PAIDOFF,1000,30,2016-09-09,2016-10-08,28,college,1,4,1
4,6,6,PAIDOFF,1000,30,2016-09-09,2016-10-08,29,college,0,4,1


## One Hot Encoding

#### How about education?


In [12]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)
df.dtypes

Unnamed: 0                 int64
Unnamed: 0.1               int64
loan_status               object
Principal                  int64
terms                      int64
effective_date    datetime64[ns]
due_date          datetime64[ns]
age                        int64
education                 object
Gender                     int64
dayofweek                  int64
weekend                    int64
dtype: object

#### Feature befor One Hot Encoding


In [13]:
df[['Principal','terms','age','Gender','education']].head()

,Principal,terms,age,Gender,education
0,1000,30,45,0,High School or Below
1,1000,30,33,1,Bechalor
2,1000,15,27,0,college
3,1000,30,28,1,college
4,1000,30,29,0,college


#### Use one hot encoding technique to conver categorical varables to binary variables and append them to the feature Data Frame


In [14]:
Feature = df[['Principal','terms','age','Gender','weekend']]
print(Feature.dtypes)
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()
Feature.isnull().sum()
Feature.dtypes


Principal    int64
terms        int64
age          int64
Gender       int64
weekend      int64
dtype: object


Principal               int64
terms                   int64
age                     int64
Gender                  int64
weekend                 int64
Bechalor                uint8
High School or Below    uint8
college                 uint8
dtype: object

In [20]:
Feature[['Principal','terms','age','Gender','weekend']]=Feature[['Principal','terms','age','Gender','weekend']].astype("float64")

### Feature selection


Lets defind feature sets, X:


In [21]:
X = Feature
X[0:5]

,Principal,terms,age,Gender,weekend,Bechalor,High School or Below,college
0,1000.0,30.0,45.0,0.0,0.0,0,1,0
1,1000.0,30.0,33.0,1.0,0.0,1,0,0
2,1000.0,15.0,27.0,0.0,0.0,0,0,1
3,1000.0,30.0,28.0,1.0,1.0,0,0,1
4,1000.0,30.0,29.0,0.0,1.0,0,0,1


What are our lables?


In [23]:
y = df['loan_status'].values
y[0:5]

array(['PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF', 'PAIDOFF'],
      dtype=object)

## Normalize Data


Data Standardization give data zero mean and unit variance (technically should be done after train test split )


In [24]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[ 0.51578458,  0.92071769,  2.33152555, -0.42056004, -1.20577805,
        -0.38170062,  1.13639374, -0.86968108],
       [ 0.51578458,  0.92071769,  0.34170148,  2.37778177, -1.20577805,
         2.61985426, -0.87997669, -0.86968108],
       [ 0.51578458, -0.95911111, -0.65321055, -0.42056004, -1.20577805,
        -0.38170062, -0.87997669,  1.14984679],
       [ 0.51578458,  0.92071769, -0.48739188,  2.37778177,  0.82934003,
        -0.38170062, -0.87997669,  1.14984679],
       [ 0.51578458,  0.92071769, -0.3215732 , -0.42056004,  0.82934003,
        -0.38170062, -0.87997669,  1.14984679]])

# Classification


Now, it is your turn, use the training set to build an accurate model. Then use the test set to report the accuracy of the model
You should use the following algorithm:

-   K Nearest Neighbor(KNN)
-   Decision Tree
-   Support Vector Machine
-   Logistic Regression

** Notice:** 

-   You can go above and change the pre-processing, feature selection, feature-extraction, and so on, to make a better model.
-   You should use either scikit-learn, Scipy or Numpy libraries for developing the classification algorithms.
-   You should include the code of the algorithm in the following cells.


# K Nearest Neighbor(KNN)

Notice: You should find the best k to build the model with the best accuracy.  
**warning:** You should not use the **loan_test.csv** for finding the best k, however, you can split your train_loan.csv into train and test to find the best **k**.


In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (242, 8) (242,)
Test set: (104, 8) (104,)


In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
#Train Model and Predict  
Ks = 10
metric_score = {}

for n in range(1,Ks):    
    #Train Model and Predict  
    Object1 = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=Object1.predict(X_test)
    metric_score[n] = metrics.accuracy_score(y_test, yhat)
    
print(metric_score)

{1: 0.7115384615384616, 2: 0.625, 3: 0.7211538461538461, 4: 0.7211538461538461, 5: 0.7307692307692307, 6: 0.7115384615384616, 7: 0.7211538461538461, 8: 0.7211538461538461, 9: 0.75}


In [27]:
Max = max(metric_score, key=metric_score.get)
print("The Key with max value:\n",Max)

The Key with max value:
 9


In [28]:
knn_object = KNeighborsClassifier(n_neighbors=9).fit(X_train,y_train)
knn_predict = knn_object.predict(X_test)
print(metrics.accuracy_score(y_test, knn_predict))

0.75


# Decision Tree


In [29]:
from sklearn.tree import DecisionTreeClassifier
tree_object = DecisionTreeClassifier(criterion="entropy", max_depth = 5).fit(X_train,y_train)
tree_predict = tree_object.predict(X_test)
print(metrics.accuracy_score(y_test, tree_predict))

0.7403846153846154


# Support Vector Machine


In [30]:
from sklearn import svm
svm_object = svm.SVC(kernel='rbf').fit(X_train,y_train)
svm_predict = svm_object.predict(X_test)
print(metrics.accuracy_score(y_test, svm_predict))

0.75


# Logistic Regression


In [31]:
from sklearn.linear_model import LogisticRegression
logistic_object = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
logistic_predict = logistic_object.predict(X_test)


In [32]:
print(metrics.accuracy_score(y_test,logistic_predict))

0.7211538461538461


# Model Evaluation using Test set


In [33]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss


First, download and load the test set:


In [45]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

--2021-03-02 23:19:34--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3642 (3.6K) [text/csv]
Saving to: ‘loan_test.csv’

loan_test.csv       100%[===================>]   3.56K  --.-KB/s    in 0s      

2021-03-02 23:19:35 (43.2 MB/s) - ‘loan_test.csv’ saved [3642/3642]



### Load Test set for evaluation


In [46]:
test_df = pd.read_csv('loan_test.csv')
test_df.head()
test_df.dtypes

Unnamed: 0         int64
Unnamed: 0.1       int64
loan_status       object
Principal          int64
terms              int64
effective_date    object
due_date          object
age                int64
education         object
Gender            object
dtype: object

In [51]:
print(test_df.isnull().sum())
print(test_df.dtypes)
test_df.shape

Unnamed: 0        0
Unnamed: 0.1      0
loan_status       0
Principal         0
terms             0
effective_date    0
due_date          0
age               0
education         0
Gender            0
dayofweek         0
weekend           0
dtype: int64
Unnamed: 0                 int64
Unnamed: 0.1               int64
loan_status               object
Principal                float64
terms                    float64
effective_date    datetime64[ns]
due_date          datetime64[ns]
age                      float64
education                 object
Gender                     int64
dayofweek                  int64
weekend                    int64
dtype: object


(54, 12)

In [48]:
test_df['due_date'] = pd.to_datetime(df['due_date'])#converstion to date
test_df['effective_date'] = pd.to_datetime(df['effective_date'])#conversion to date time format
test_df['Principal'] = test_df['Principal'].astype('float64')
test_df['age'] = test_df['age'].astype('float64')
test_df['terms'] = test_df['terms'].astype('float64') 



test_df['dayofweek'] = df['effective_date'].dt.dayofweek #extracting day of week
test_df['weekend'] = df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)  #eaxtracting weekend
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True) #Converting Gender to label encoder







In [50]:
print(test_df.isnull().sum())
print(test_df.shape)


Unnamed: 0        0
Unnamed: 0.1      0
loan_status       0
Principal         0
terms             0
effective_date    0
due_date          0
age               0
education         0
Gender            0
dayofweek         0
weekend           0
dtype: int64
(54, 12)


In [38]:
kolaveriD = test_df[['Principal','terms','age','Gender','weekend']] # Extracting features as trained above from data set
kolaveriD = pd.concat([kolaveriD,pd.get_dummies(test_df['education'])], axis=1) # one hot encoding to education column
kolaveriD.drop(['Master or Above'], axis = 1,inplace=True) # dropping master column because 50% yes 50% no cannot be a predictpr
#X = Feature
#y = df['loan_status'].values
#X= preprocessing.StandardScaler().fit(X).transform(X)
#X[0:5]
#X_test.head()


In [39]:
#X_test = X_test.drop(X_test.columns[[9,10]], axis=1)
print(kolaveriD.head())
print(kolaveriD.dtypes)

   Principal  terms   age  Gender  weekend  Bechalor  High School or Below  \
0     1000.0   30.0  50.0       1        0         1                     0   
1      300.0    7.0  35.0       0        0         0                     0   
2     1000.0   30.0  43.0       1        0         0                     1   
3     1000.0   30.0  26.0       0        1         0                     0   
4      800.0   15.0  29.0       0        1         1                     0   

   college  
0        0  
1        0  
2        0  
3        1  
4        0  
Principal               float64
terms                   float64
age                     float64
Gender                    int64
weekend                   int64
Bechalor                  uint8
High School or Below      uint8
college                   uint8
dtype: object


In [40]:
#X.drop(X[['Bechalor','High School or Below','college']], inplace = True, axis = 1)
#X=set(X)
#X=pd.DataFrame(X)
kolaveriD[['Gender','weekend']] = kolaveriD[['Gender','weekend']].astype("float64")

In [54]:
kolaveriD[['Bechalor','High School or Below','college']] = kolaveriD[['Bechalor','High School or Below','college']].astype("float64")

In [55]:
kolaveriD.isnull().sum()

Principal               0
terms                   0
age                     0
Gender                  0
weekend                 0
Bechalor                0
High School or Below    0
college                 0
dtype: int64

In [56]:
X_men = kolaveriD
X_men= preprocessing.StandardScaler().fit(X_men).transform(X_men)
X_men[0:5]



array([[ 0.49362588,  0.92844966,  3.05981865,  1.97714211, -4.12310563,
         2.39791576, -0.79772404, -0.86135677],
       [-3.56269116, -1.70427745,  0.53336288, -0.50578054, -4.12310563,
        -0.41702883, -0.79772404, -0.86135677],
       [ 0.49362588,  0.92844966,  1.88080596,  1.97714211, -4.12310563,
        -0.41702883,  1.25356634, -0.86135677],
       [ 0.49362588,  0.92844966, -0.98251057, -0.50578054,  0.24253563,
        -0.41702883, -0.79772404,  1.16095912],
       [-0.66532184, -0.78854628, -0.47721942, -0.50578054,  0.24253563,
         2.39791576, -0.79772404, -0.86135677]])

In [57]:
X_men.shape

(54, 8)

In [58]:
y_men = test_df['loan_status'].values

In [65]:
from sklearn.metrics import log_loss
log_predict = logistic_object.predict(X_men)
log_prob = logistic_object.predict_proba(X_men)
print("f1 score is :", f1_score(y_men, log_predict, average='weighted'))
print("log loss is :", log_loss(y_men, log_prob))
print("jaccard accuracy score is: %2f", jaccard_score(y_men, log_predict,pos_label='PAIDOFF', average='binary'))

f1 score is : 0.6304176516942475
log loss is : 0.6081218415352306
jaccard accuracy score is: %2f 0.7407407407407407


In [66]:
knn_predict = knn_object.predict(X_men)
print("f1 score is :", f1_score(y_men, knn_predict, average='weighted'))
print("jaccard accuracy score is: %2f", jaccard_score(y_men, knn_predict,pos_label='PAIDOFF', average='binary'))

f1 score is : 0.5925925925925926
jaccard accuracy score is: %2f 0.6666666666666666


In [67]:
tree_predict = tree_object.predict(X_men)
print("f1 score is :", f1_score(y_men, tree_predict, average='weighted'))
print("jaccard accuracy score is: %2f", jaccard_score(y_men, tree_predict,pos_label='PAIDOFF', average='binary'))

f1 score is : 0.6717642373556352
jaccard accuracy score is: %2f 0.7547169811320755


In [68]:
svm_predict = svm_object.predict(X_men)
print("f1 score is :", f1_score(y_men, svm_predict, average='weighted'))
print("jaccard accuracy score is: %2f", jaccard_score(y_men, svm_predict,pos_label='PAIDOFF', average='binary'))

f1 score is : 0.6304176516942475
jaccard accuracy score is: %2f 0.7407407407407407


In [72]:
report_dict ={} 
  
# Insert data into dicitonary 
dict1 = {'Algorithm':['F1-score','Jaccard_score','log-loss'],
    'log': ["0.6304176516942475",'0.7407407407407407', '0.6081218415352306'],
    'knn': ["59.259", "0.666", 'NA'], 
    'svm': ["0.630", "0.740", 'NA'],  
    'tree': ["0.671","0.75","NA"]} 
  
# Print the names of the columns. 
#print ("{:<10} {:<10} {:<10}".format('F1', 'Jaccrd', 'log-loss')) 
  
# print each data item. 
for key, value in dict1.items(): 
    algorithm = key
    F1, Jaccrd, log_loss = value 
    print ("{} {:<10} {:<10} {:<10}".format(algorithm,F1, Jaccrd, log_loss)) 

Algorithm F1-score   Jaccard_score log-loss  
log 0.6304176516942475 0.7407407407407407 0.6081218415352306
knn 59.259     0.666      NA        
svm 0.630      0.740      NA        
tree 0.671      0.75       NA        


# Report

You should be able to report the accuracy of the built model using different evaluation metrics:


| Algorithm          | Jaccard | F1-score | LogLoss |
| ------------------ | ------- | -------- | ------- |
| KNN                | ?       | ?        | NA      |
| Decision Tree      | ?       | ?        | NA      |
| SVM                | ?       | ?        | NA      |
| LogisticRegression | ?       | ?        | ?       |


<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="http://cocl.us/ML0101EN-SPSSModeler">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://cocl.us/ML0101EN_DSX">Watson Studio</a>

<h3>Thanks for completing this lesson!</h3>

<h4>Author:  <a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a></h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<hr>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description                                                             |
| ----------------- | ------- | ------------- | ------------------------------------------------------------------------------ |
| 2020-10-27        | 2.1     | Lakshmi Holla | Made changes in import statement due to updates in version of  sklearn library |
| 2020-08-27        | 2.0     | Malika Singla | Added lab to GitLab                                                            |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>

<p>
